In [ ]:
import gzip
from lxml import etree

In [ ]:
input_file_path = "pubmed20n0471.xml.gz"

In [ ]:
with gzip.open(input_file_path, "rb") as input_file:
    tree = etree.parse(input_file)

In [ ]:
root = tree.getroot()

In [ ]:
for i, element in enumerate(root.iter()):
    print("%s - %s" % (element.tag, element.text))
    if i >= 100:
        break